# Usage of Package

### Import package in Notebook 
To activate enviroment from this folder and use `ConnectedInformation` package 
run following commands:

In [2]:
using Pkg
Pkg.activate("../")
using ConnectedInformation

  Activating project at `~/Homework/Bachelor thesis/ConnectedInformation`


For examples we will calculate entropies for constraints from given
distribution probabilities using Entropy module.

In [3]:
include("../src/entropy.jl")
using .Entropy: caluclate_all_entropies

### Estimating connected information
Function `estimate_connected_information(k, distrs_card, entropy_constraints)` 
is used to estimate connected information, where
- `k` is order of connected information needed to estiamte
- `distrs_card` is vector of all distribution cardinalities
- `entropy_constraints` is dictionary of constraints on entropies 

##### Parity example
$Y = X_1 \oplus X_2 \oplus X_3$
| $X_1$ | $X_2$ | $X_3$ | $Y$ | $p$ |
|---|---|---|---|---|
| 0 | 0 | 0 | 0 | 1/8 |
| 0 | 0 | 1 | 1 | 1/8 |
| 0 | 1 | 0 | 1 | 1/8 |
| 0 | 1 | 1 | 0 | 1/8 |
| 1 | 0 | 0 | 1 | 1/8 |
| 1 | 0 | 1 | 0 | 1/8 |
| 1 | 1 | 0 | 0 | 1/8 |
| 1 | 1 | 1 | 1 | 1/8 |


In [4]:
parity = zeros(Float64, 2, 2, 2, 2)
parity[1, 1, 1, 1] = 1/8
parity[1, 1, 2, 2] = 1/8
parity[1, 2, 1, 2] = 1/8
parity[1, 2, 2, 1] = 1/8
parity[2, 1, 1, 2] = 1/8
parity[2, 1, 2, 1] = 1/8
parity[2, 2, 1, 1] = 1/8
parity[2, 2, 2, 2] = 1/8

entropies = caluclate_all_entropies(parity)        
distr_cards = [2, 2, 2, 2]
connected_information = estimate_connected_information(4, distr_cards, entropies)

4-element Vector{Float64}:
 NaN
   0.0
   0.0
   1.0

##### Random values example

In [9]:
distr_cards = [3, 2, 4]
rnd_values = Array{Float64,3}(undef, distr_cards...)
rnd_values[:,:,1] = [0.0732422  0.0258789;
                     0.124512   0.0473633;
                     0.0322266  0.000976562]
rnd_values[:,:,2] = [0.0102539  0.00732422;
                     0.0317383  0.120605;
                     0.0791016  0.0595703;]
rnd_values[:,:,3] = [0.00390625  0.0249023;
                     0.0292969   0.00927734;
                     0.0356445   0.0893555]
rnd_values[:,:,4] = [0.0209961  0.0439453;
                     0.0839844  0.0219727;
                     0.0136719  0.0102539]
rnd_values

3×2×4 Array{Float64, 3}:
[:, :, 1] =
 0.0732422  0.0258789
 0.124512   0.0473633
 0.0322266  0.000976562

[:, :, 2] =
 0.0102539  0.00732422
 0.0317383  0.120605
 0.0791016  0.0595703

[:, :, 3] =
 0.00390625  0.0249023
 0.0292969   0.00927734
 0.0356445   0.0893555

[:, :, 4] =
 0.0209961  0.0439453
 0.0839844  0.0219727
 0.0136719  0.0102539

In [10]:
entropies = caluclate_all_entropies(rnd_values)
connected_information = estimate_connected_information(3, distr_cards, entropies)

3-element Vector{Float64}:
 NaN
   0.20138705009070534
   0.17931841065010445


### Estimating maximum entropy
Funtion `estimate_max_enthropy(k, distrs_card, entropy_constraints)` is used to 
estimate maximum entropy, where
- `k` is order of entropy
- `distrs_card` is vector with all distribution cardinalities
- `entropy_constraints` is dictionary of constraints on entropies 

In [7]:
distr_cards = [2, 2, 2, 2]

parity = zeros(Float64, 2, 2, 2, 2)
parity[1, 1, 1, 1] = 1/8
parity[1, 1, 2, 2] = 1/8
parity[1, 2, 1, 2] = 1/8
parity[1, 2, 2, 1] = 1/8
parity[2, 1, 1, 2] = 1/8
parity[2, 1, 2, 1] = 1/8
parity[2, 2, 1, 1] = 1/8
parity[2, 2, 2, 2] = 1/8
parity_entropies = caluclate_all_entropies(parity)

estimate_max_enthropy(4, distr_cards, parity_entropies)

3.0

##### Result inaccuracy
 Sometimes function can return lower value than maximum possible entropy. 
 In example below maximum possible entropy is 4, however, function returns 
 a little smaller number. 

In [8]:
distr_cards = [2, 2, 2, 2]
one_dim_entropies = Dict{Vector{Int64}, Float64}([1] => 1, [2] => 1, [3] => 1, [4] => 1)
estimate_max_enthropy(1, distr_cards, one_dim_entropies)


3.9999999999996314

Since `estimate_max_enthropy` function is used as part of connected information 
calculation, it is important to take this inaccuarcy into account.   
Therefore, if entropy of the next order turns out to have larger value inside 
`estimate_connected_information` function due to estimation inaccuracy, 
then connected information of the next order is set to 0.